## BETTER WAY 15 - 클로저 변수 스코프와 작용하는 방법을 알자
## Item15 - Know How Closures Interact with Variable Scope

숫자 리스트를 정렬할 때 특정 그룹의 숫자들이 먼저 오도록 우선순위를 매기려고 할때, 아래의 예제처럼 리스트의 `sort()`메소드에 헬퍼함수를 `key`인수로 넘기는 방법이 있다. 헬퍼의 반환값(여기서는 튜플)은 리스트에 있는 각 아이템을 정렬하는 값으로 사용된다.

In [3]:
def sort_priority(values, group):
    def helper(x):
        if x in group:
            print((0, x))
            return (0, x)
        print((1, x))
        return (1, x)
    values.sort(key=helper)

In [4]:
numbers = [8, 3, 1, 2, 5, 4, 7, 6]
group = {2, 3, 5, 7}
sort_priority(numbers, group)
print(numbers)

(1, 8)
(0, 3)
(1, 1)
(0, 2)
(0, 5)
(1, 4)
(0, 7)
(1, 6)
[2, 3, 5, 7, 1, 4, 6, 8]


위의 코드를 보고 함수안에 함수를 쓰는 것을 처음 보았다... 이러한 개념을 이해하려면 먼저, 파이썬의 **퍼스트클래스 함수(First-class function)**에 대해 알아야 한다. [School of web](http://schoolofweb.net/blog/posts/%ED%8C%8C%EC%9D%B4%EC%8D%AC-%ED%8D%BC%EC%8A%A4%ED%8A%B8%ED%81%B4%EB%9E%98%EC%8A%A4-%ED%95%A8%EC%88%98-first-class-function/)을 참고하였다. <br />
**퍼스트클래스 함수**란 프로그래밍 언어가 함수(function)를 *first-class object* 로 취급하는 것을 의미한다. 즉, 함수 자체를 **인자(argument)**로써 다른 함수에 전달하거나 다른 함수의 결과값으로 **리턴할 수** 있고, 함수를 변수에 **할당**하거나 데이터 구조안에 **저장**할 수 있는 함수를 말한다. 아래의 예제 코드들을 통해 퍼스트클래스 함수에 대해 자세히 알아보도록 하자.

In [1]:
def square(x):
    return x * x

print(square(5))

f = square

print(square)
print(f)

25
<function square at 0x10fc04e18>
<function square at 0x10fc04e18>


위의 코드를 보면 `square`함수와 `f` 변수에 `square`를 할당한 후 출력한 결과, 같은 메모리에 할당되어 있는것을 볼 수 있다. 또한, 아래의 코드를 통해 `square`함수를 할당받은 `f`변수가 함수처럼 사용되는 것을 확인할 수 있다.

In [2]:
def square(x):
    return x * x

f = square

print(f(5))

25


이번에는 함수 자체를 인자(argument)로써 다른 함수에 전달하는 코드를 살펴보겠다. my_map 함수에 `square`함수를 인자로 전달한 후 `for`루프안에서 `square`함수를 호출하는 것을 볼 수 있다. 

In [3]:
def square(x):
    return x * x

def my_map(func, arg_list):
    result = []
    for i in arg_list:
        result.append(func(i))  # square 함수 호출
    return result

num_list = [1, 2, 3, 4, 5]
squares = my_map(square, num_list)  # square 함수를 인자로 전달
print(squares)

[1, 4, 9, 16, 25]
